# Basketball (or Netball) Cannon Toy Problem

Develop a controller for a basketball cannon.  Given the following inputs
1. distance from the basket
2. height from which the ball leaves the cannon
3. speed the ball will leave the cannon

**What angle should the cannon aim the ball such that it will go through the basket without interacting with the backboard or the rim.**

Note that this scenario in actually more similar to Netball than Basketball, since we are neglecting the backboard.
<img src="https://netballamerica.com/wp-content/uploads/1DX28219.jpg">

In [1]:
# import some basic packages
import itertools
import numpy as np
from collections import defaultdict
from typing import Tuple

In [68]:
np.set_printoptions(suppress=True)

In [2]:
from bokeh import plotting
from bokeh.palettes import Dark2_8 as palette
colors = itertools.cycle(palette)

In [3]:
plotting.output_notebook()

Loading BokehJS ...

In this problem, we observe that the parameters are such that cannon is always aimed in the vertical plane connecting the cannon and the hoop. We can separate the motion of the ball into the horizontal, $x$, and vertical, $y$, directions.  The position in both directions can be written using the linear equations of motions as follows.

$$x(t) = x_0 + \dot{x}_0 t + \frac{1}{2} \ddot{x}_0 t^2$$

$$y(t) = y_0 + \dot{y}_0 t + \frac{1}{2} \ddot{y}_0 t^2$$

We can set the x and y origin at the base of the cannon, so the ball leaves the cannon at $(x_0, y_0) = (0, h_0)$.  The speed at which the ball leaves the cannon is written in terms of x and y as $(\dot{x}_0, \dot{y}_0) = (v\cos\theta, v\sin\theta)$.  The only acceleration in this situation is gravity so $(\ddot{x}_0, \ddot{y}_0) = (0, g)$.  Based on these values, the equations of motion can be written as

$$x(t) = v\cos(\theta) t$$

$$y(t) = h_0 + v\sin(\theta) t + \frac{1}{2}g t^2$$


To measure how close each shot gets to the basket, we want to consider the time when the ball passes the height of the basket, $h_{b}$, for the second time.  

$$\frac{1}{2} g t^2 + v \sin(\theta) t + \delta_h = 0$$

where $\delta_h$ is the difference in height, $\delta_h = h_0 - h_{basket}$.  This occurs when 

$$t = \frac{-v \sin(\theta) \pm \sqrt{v^2 \sin^2(\theta)+2gh}}{g}$$

Note that in order to have a reasonable throw, using the assumption that the cannon is always below the rim, the trajectory of the ball must form an arch passing height of the rim twice.  This only occurs when the descriminant is positive, which means

$$v^2 > \frac{-2 g h}{sin^2(\theta)}$$

Lets begin by defining a cannon class that expects the three inputs, distance, height, and speed.  We also define the `fire` method which takes an angle as input then provides as output the distance and impact angle the ball will make with the horizontal plane of the hoop.  We also define a `show_path` method which plots all `fire` attempts.

Note that we are using a ball width of 24 cm, and a rim that is twice that width.  This means that the ball goes through the rim if the center of the ball is within half the ball width of the center of the rim.

In [4]:
class Cannon():
    def __init__(self, dist: float, height: float, speed: float) -> None: 
        # constants
        self.ball_radius = 0.24 / 2 # m
        self.height_basket = 3.05  # m
        self.g = -9.8  # m/s^2
        
        # cannon parameters
        self.dist = dist  # m, distance to hoop
        self.height = height  # m, height of the cannon
        self.speed = speed  # m/s
        
        self.delta_h = self.height_basket - self.height
        self.rim = dist + self.ball_radius * np.array([-1, 1])  # m

        self.path = []
        self.angle = 0
        
    def fire(self, angle: float):
        self.angle = np.deg2rad(angle)
        cos = np.cos(self.angle)
        sin = np.sin(self.angle)

        self.result = 'terrible shot'
        D = (self.speed * sin) ** 2 + 2 * self.g * self.delta_h
        if D > 0:  # ball passes through correct height twice
            self.t_m = (-self.speed * sin - np.sqrt(D)) / self.g  # negative root is the downward part of the arc
            self.x_shot = self.speed * cos * self.t_m  
            self.delta_x = self.x_shot - self.dist  # neg: undershot, pos: overshot
            if self.rim[0] < self.x_shot < self.rim[1]:
                self.result = 'success'
            else:
                self.result = 'miss'
            t_final = self.t_m * 1.1
        else:
            # TODO: consider this case further
            self.delta_x = 999  # high penalty
            t_final = 1.5  # s

        t = np.linspace(0, t_final, 100)  # 0 to 10 s
        x = self.speed * cos * t
        y = self.height + self.speed * sin * t + self.g * t**2 / 2
        self.path.append(np.c_[x, y][y > 0])
        
        return self.delta_x
    
    def _dx(self, angle):
        sin = np.sin(angle)
        cos = np.cos(angle)
        D = (self.speed * sin) ** 2 + 2 * self.g * self.delta_h
        sqrt_D = np.sqrt(D)
        dx = (self.speed ** 2 / self.g * (sin ** 2 - cos ** 2) 
              - self.speed * sin / self.g * (-sqrt_D + (self.speed * cos) ** 2 / sqrt_D))
        return dx        
        
    def show_path(self):
        if self.path:
            p = plotting.figure(
                width=800, height=400, match_aspect=True,
                x_axis_label='(meters)', y_axis_label='(meters)',
                x_range=(-1, self.dist*1.3), y_range=(-0.5, 6),
            )

            # plot the hoop
            p.line(np.array([1, 1.015, 1.015]) * self.rim[1], 
                   [self.height_basket, self.height_basket, 0], color='gray')
            p.circle(self.rim, self.height_basket,  
                     radius=0.02, color='orange')
            p.line(self.rim, self.height_basket, color='orange')

            # plot the attempts
            for i, (path, color) in enumerate(zip(self.path, colors)):
                name = f'Attempt {i+1}'
                p.line(path[:, 0], path[:, 1], color=color, 
                       legend_label=name, muted_alpha=0.2)
                t = np.linspace(0, 1, 100)
                
            # plot the last cannon
            p.line(0, [0, self.height], color='black')
            p.line(
                [-np.cos(self.angle)/2, 0], 
                [self.height - np.sin(self.angle)/2, self.height],
                line_width=5, color=color,
            )

            p.legend.location = 'top_right'
            p.legend.click_policy = 'mute'
            p.toolbar.autohide = True
            plotting.show(p)
        else:
            print('Error: no attempts made!')

Lets look at a cannon placed roughly at the free-throw line, 5 meters from the basket, shot from 2 meters above the ground, and fired at 8 meters per second.

In [5]:
freethrow = Cannon(5, 2, 8)

for angle in [45, 40, 43, 60]:
    delta_x = freethrow.fire(angle)
    print(f'{angle} degrees: {freethrow.result} ({delta_x:0.2f} m away)')

freethrow.show_path()

45 degrees: miss (0.22 m away)
40 degrees: miss (-0.27 m away)
43 degrees: success (0.07 m away)
60 degrees: success (-0.03 m away)


We see that the first shot, at an angle of 45 degrees, was 0.22 meters past the basket.
The second shot, at an angle of 40 degrees, was 0.27 meters short of the basket.
The third shot, at 43 degrees went through the basket.
Maybe surprising, the fourth shot, at 60 degrees, also went through the basket.

Lets position the cannon roughtly at the three-point line, 7 meters from the basket.  We will use the same height, but increase the velocity a bit as we are a bit farther away.

In [6]:
three_point = Cannon(7, 2, 9)

three_point.fire(45)  # Rachael's first attempt
three_point.fire(54)  # 2nd angle
three_point.show_path()

How do we figure out the angle for other situations?  We could always brute force a solution.

In [7]:
c3 = Cannon(15, 2, 15)
res = []
for angle in range(20, 75):
    res.append(c3.fire(angle))

c3.show_path()

## Learn what angle to fire the cannon

A more interesting approach is to create an agent that can explore firing this cannon to experimentally learn what angle to aim the cannon to successfully fire the ball into the hoop.

The goal for our agent, or cost function we want to minimize, is the distance the ball passes from the center of the basket (`delta_x`).

In [43]:
class Agent():
    def __init__(self, cannon, *, learning_rate=0.5):
        self.cannon = cannon
        self.lr = learning_rate
#         self.epsilon_decay = epsilon_decay
        self.rng = np.random.default_rng()
    
    @staticmethod
    def _grad_delta_x(results, i):
        # this is not the best as it is only one-sided
        return (results[i, 1] - results[i-1, 1]) / (results[i, 0] - results[i-1, 0])
    
    def learn(self, max_iters, *, angle_0=89):
        # hacky gradient descent
        
        results = np.empty((max_iters, 2))
        
        delta_x_0 = self.cannon.fire(angle_0)
        results[0, :] = angle_0, delta_x_0
        
        if angle_0 < 45:
            angle_1 = angle_0 + 1
        else:
            angle_1 = angle_0 -1
            
        delta_x_1 = self.cannon.fire(angle_1)
        results[1, :] = angle_1, delta_x_1
        
        angle_ip1 = angle_1 - self.lr * self._grad_delta_x(results, 1)
        
        for i in range(2, max_iters):
            angle_i = angle_ip1
            delta_x = self.cannon.fire(angle_i)
            results[i, :] = angle_i, delta_x
            if np.abs(delta_x) < self.cannon.ball_radius:
                break
            angle_ip1 = angle_i +  self.lr * self._grad_delta_x(results, i)
        
        self.results = results[:i+1]

Test the agent with a cannon that is 15 meters away, 2 meters off the ground, and firing at a speed of 15 meters / second.

Use an initial guess of 20 degrees.

In [44]:
c3 = Cannon(15, 2, 15)
johnny5 = Agent(c3)
johnny5.learn(100, angle_0=20)

In [54]:
johnny5.results

array([[20.        , -4.17512663],
       [21.        , -3.19787242],
       [19.53411868, -4.67945437],
       [21.05018474, -3.15176349],
       [22.56168636, -1.85949126],
       [23.84412518, -0.87804946],
       [24.99206509, -0.06908656]])

In [46]:
c3.show_path()

In [47]:
len(johnny5.results)

14

The agent found the correct angle in 14 trials.

Lets increase the learning rate.  We really want the steps to be at least 1 degree.

In [50]:
c3 = Cannon(15, 2, 15)
johnny5 = Agent(c3, learning_rate=1.5)
johnny5.learn(100, angle_0=20)

In [55]:
johnny5.results

array([[20.        , -4.17512663],
       [21.        , -3.19787242],
       [19.53411868, -4.67945437],
       [21.05018474, -3.15176349],
       [22.56168636, -1.85949126],
       [23.84412518, -0.87804946],
       [24.99206509, -0.06908656]])

In [52]:
c3.show_path()

In [53]:
len(johnny5.results)

7

Now it found a valid angle in only 7 steps.


Lets use the same cannon with a initial angle of 80 degrees.

In [70]:
c3 = Cannon(15, 2, 15)
johnny5 = Agent(c3, learning_rate=1.5)
johnny5.learn(100, angle_0=80)

In [71]:
johnny5.results

array([[80.        , -7.33722412],
       [79.        , -6.60852564],
       [80.09304772, -7.40553522],
       [78.99930373, -6.60802186],
       [77.90556511, -5.82321884],
       [76.82925263, -5.06448386],
       [75.77184366, -4.33326758],
       [74.73456816, -3.63065544],
       [73.71852352, -2.95745837],
       [72.72467387, -2.31422311],
       [71.75385008, -1.70124587],
       [70.80675147, -1.11858927],
       [69.88394903, -0.56610178],
       [68.98588982, -0.04343892]])

In [72]:
c3.show_path()

In [73]:
c3 = Cannon(15, 2, 15)
johnny5 = Agent(c3, learning_rate=0.5)
johnny5.learn(1000, angle_0=45)

/home/schowell/data/myPrograms/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/schowell/data/myPrograms/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in greater


In [74]:
johnny5.results[500:600]

array([[46.37207116,  6.8840085 ],
       [46.37209203,  6.8840085 ],
       [46.37211259,  6.8840085 ],
       [46.37213284,  6.8840085 ],
       [46.3721528 ,  6.8840085 ],
       [46.37217246,  6.8840085 ],
       [46.37219182,  6.8840085 ],
       [46.3722109 ,  6.8840085 ],
       [46.3722297 ,  6.8840085 ],
       [46.37224822,  6.88400851],
       [46.37226646,  6.88400851],
       [46.37228444,  6.88400851],
       [46.37230214,  6.88400851],
       [46.37231959,  6.88400851],
       [46.37233677,  6.88400851],
       [46.37235371,  6.88400851],
       [46.37237039,  6.88400851],
       [46.37238682,  6.88400851],
       [46.37240301,  6.88400851],
       [46.37241896,  6.88400851],
       [46.37243468,  6.88400851],
       [46.37245016,  6.88400851],
       [46.37246541,  6.88400851],
       [46.37248043,  6.88400851],
       [46.37249524,  6.88400851],
       [46.37250982,  6.88400851],
       [46.37252419,  6.88400851],
       [46.37253834,  6.88400851],
       [46.37255229,

The gradient can get stuck if it starts between the two possible solutions.

In [75]:
p = plotting.figure()
p.line(johnny5.results[:, 0], johnny5.results[:, 1])
plotting.show(p)

In [76]:
p = plotting.figure()
i = np.arange(len(johnny5.results[:, 0]))
p.line(i, johnny5.results[:, 0], color='green')
p.line(i, johnny5.results[:, 1])
plotting.show(p)